In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn import metrics
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression

In [2]:
data=pd.read_csv('train.csv')
test=pd.read_csv("test.csv")
datadiff=pd.read_csv("extradata.csv")
testdiff=pd.read_csv("extratest.csv")
data=pd.concat([data,datadiff.iloc[:,1:3]],axis=1)
test=pd.concat([test,testdiff.iloc[:,1:3]],axis=1)

In [3]:
df43 = pd.DataFrame(data, columns = ['genre_id', 'is_listened'])
df45 = pd.DataFrame(data, columns = ['media_id', 'is_listened'])
df46=  pd.DataFrame(data, columns = ['album_id', 'is_listened'])
df47=  pd.DataFrame(data, columns = ['artist_id', 'is_listened'])
df51=  pd.DataFrame(data, columns = ['user_id', 'is_listened'])
df53=  pd.DataFrame(data, columns = ['release_date', 'is_listened'])
df55=  pd.DataFrame(data, columns = ['time1', 'is_listened'])

In [4]:
#TO create fractional features. First groupby ids and then calculate mean of is listened for every level of id
#and then map it on data using dictionary creation
count1=df43.groupby(['genre_id'],as_index=False).agg({'is_listened':'mean'})
df44=pd.Series(count1.is_listened.values,index=count1.genre_id).to_dict()
data["B"] = data["genre_id"].map(df44)
test["B"] = test["genre_id"].map(df44)


count2=df45.groupby(['media_id'],as_index=False).agg({'is_listened':'mean'})
df48=pd.Series(count2.is_listened.values,index=count2.media_id).to_dict()
data["C"] = data["media_id"].map(df48)
test["C"] = test["media_id"].map(df48)

count3=df46.groupby(['album_id'],as_index=False).agg({'is_listened':'mean'})
df49=pd.Series(count3.is_listened.values,index=count3.album_id).to_dict()
data["D"] = data["album_id"].map(df49)
test["D"] = test["album_id"].map(df49)

count4=df47.groupby(['artist_id'],as_index=False).agg({'is_listened':'mean'})
df50=pd.Series(count4.is_listened.values,index=count4.artist_id).to_dict()
data["E"] = data["artist_id"].map(df50)
test["E"] = test["artist_id"].map(df50)

count5=df51.groupby(['user_id'],as_index=False).agg({'is_listened':'mean'})
df52=pd.Series(count5.is_listened.values,index=count5.user_id).to_dict()
data["F"] = data["user_id"].map(df52)
test["F"] = test["user_id"].map(df52)

count6=df53.groupby(['release_date'],as_index=False).agg({'is_listened':'mean'})
df54=pd.Series(count6.is_listened.values,index=count6.release_date).to_dict()
data["G"] = data["release_date"].map(df54)
test["G"] = test["release_date"].map(df54)

count7=df55.groupby(['time1'],as_index=False).agg({'is_listened':'mean'})
df56=pd.Series(count7.is_listened.values,index=count7.time1).to_dict()
data["H"] = data["time1"].map(df56)
test["H"] = test["time1"].map(df56)

In [5]:
#pivot table for clustering of similar users. Till now only used genre id for cluster
matrix = data.pivot_table(index=['user_id'], columns=['genre_id'], values='is_listened')
from sklearn.cluster import KMeans
cluster = KMeans(n_clusters=70)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usergencluster"] = data["user_id"].map(mat2)
test["usergencluster"]=test["user_id"].map(mat2)

data['album_id_1']=data['album_id']
test['album_id_1']=test['album_id']
mmo3=pd.DataFrame(data['album_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.album_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[1500:,0])
data['album_id_1'].loc[data['album_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['album_id_1'], values='is_listened')
cluster = KMeans(n_clusters=100)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["useralbcluster"] = data["user_id"].map(mat2)
test["useralbcluster"]=test["user_id"].map(mat2)

data['media_id_1']=data['media_id']
test['media_id_1']=test['media_id']
mmo3=pd.DataFrame(data['media_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.media_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[3000:,0])
data['media_id_1'].loc[data['media_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['media_id_1'], values='is_listened')
cluster = KMeans(n_clusters=100)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usermedcluster"] = data["user_id"].map(mat2)
test["usermedcluster"]=test["user_id"].map(mat2)

data['artist_id_1']=data['artist_id']
test['artist_id_1']=test['artist_id']
mmo3=pd.DataFrame(data['artist_id_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.artist_id_1.values)],axis=1)
mmo31=list(mmo3.iloc[1700:,0])
data['artist_id_1'].loc[data['artist_id_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['artist_id_1'], values='is_listened')
cluster = KMeans(n_clusters=80)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userartcluster"] = data["user_id"].map(mat2)
test["userartcluster"]=test["user_id"].map(mat2)

matrix=0
data['release_date_1']=data['release_date']
test['release_date_1']=test['release_date']
mmo3=pd.DataFrame(data['release_date_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.release_date_1.values)],axis=1)
mmo31=list(mmo3.iloc[1000:,0])
data['release_date_1'].loc[data['release_date_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['release_date_1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userdatecluster"] = data["user_id"].map(mat2)
test["userdatecluster"]=test["user_id"].map(mat2)

matrix=0
data['time1_1']=data['time1']
test['time1_1']=test['time1']
mmo3=pd.DataFrame(data['time1_1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.time1_1.values)],axis=1)
mmo31=list(mmo3.iloc[:,0])
#data['time1_1'].loc[data['time1_1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['time1_1'], values='is_listened')
cluster = KMeans(n_clusters=4)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usertimecluster"] = data["user_id"].map(mat2)
test["usertimecluster"]=test["user_id"].map(mat2)

matrix=0
data['user_age1']=data['user_age']
test['user_age1']=test['user_age']
mmo3=pd.DataFrame(data['user_age1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.user_age1.values)],axis=1)
mmo31=list(mmo3.iloc[1000:,0])
data['user_age1'].loc[data['user_age1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['user_age1'], values='is_listened')
cluster = KMeans(n_clusters=4)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["useragecluster"] = data["user_id"].map(mat2)
test["useragecluster"]=test["user_id"].map(mat2)

matrix=0
data['context_type1']=data['context_type']
test['context_type1']=test['context_type']
mmo3=pd.DataFrame(data['context_type1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.context_type1.values)],axis=1)
mmo31=list(mmo3.iloc[1000:,0])
data['context_type1'].loc[data['context_type1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['context_type1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usercontcluster"] = data["user_id"].map(mat2)
test["usercontcluster"]=test["user_id"].map(mat2)

matrix=0
data['platform_name1']=data['platform_name']
test['platform_name1']=test['platform_name']
mmo3=pd.DataFrame(data['platform_name1'].value_counts())
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.platform_name1.values)],axis=1)
mmo31=list(mmo3.iloc[1000:,0])
data['platform_name1'].loc[data['platform_name1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['platform_name1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userplatcluster"] = data["user_id"].map(mat2)
test["userplatcluster"]=test["user_id"].map(mat2)

matrix=0
#data['platform_name1']=data['platform_name']
#test['platform_name1']=test['platform_name']
#mmo3=pd.DataFrame(data['platform_name1'].value_counts())
#mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.platform_name1.values)],axis=1)
#mmo31=list(mmo3.iloc[1000:,0])
#data['platform_name1'].loc[data['platform_name1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['user_gender'], values='is_listened')
cluster = KMeans(n_clusters=6)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["usergendcluster"] = data["user_id"].map(mat2)
test["usergendcluster"]=test["user_id"].map(mat2)

matrix=0
matrix = data.pivot_table(index=['genre_id'], columns=['user_age'], values='is_listened')
cluster = KMeans(n_clusters=5)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[0:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['genre_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.genre_id.values).to_dict()
data["genreagecluster"] = data["genre_id"].map(mat2)
test["genreagecluster"]=test["genre_id"].map(mat2)

matrix = data.pivot_table(index=['genre_id'], columns=['context_type1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[0:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['genre_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.genre_id.values).to_dict()
data["genrecontcluster"] = data["genre_id"].map(mat2)
test["genrecontcluster"]=test["genre_id"].map(mat2)

matrix=0
data['listen_type1']=data['listen_type']
test['listen_type1']=test['listen_type']

matrix = data.pivot_table(index=['genre_id'], columns=['listen_type1'], values='is_listened')
cluster = KMeans(n_clusters=5)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[0:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['genre_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.genre_id.values).to_dict()
data["genrelistcluster"] = data["genre_id"].map(mat2)
test["genrelistcluster"]=test["genre_id"].map(mat2)

matrix = data.pivot_table(index=['genre_id'], columns=['user_gender'], values='is_listened')
cluster = KMeans(n_clusters=5)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[0:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['genre_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.genre_id.values).to_dict()
data["genregendcluster"] = data["genre_id"].map(mat2)
test["genregendcluster"]=test["genre_id"].map(mat2)

matrix = data.pivot_table(index=['genre_id'], columns=['user_age'], values='is_listened')
cluster = KMeans(n_clusters=4)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[0:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['genre_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.genre_id.values).to_dict()
data["genreagecluster"] = data["genre_id"].map(mat2)
test["genreagedcluster"]=test["genre_id"].map(mat2)


#mmo3=pd.DataFrame(data['platform_name1'].value_counts())
#mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.platform_name1.values)],axis=1)
#mmo31=list(mmo3.iloc[1000:,0])
#data['platform_name1'].loc[data['platform_name1'].isin(mmo31)]=-1
matrix = data.pivot_table(index=['user_id'], columns=['listen_type1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userlisttypecluster"] = data["user_id"].map(mat2)
test["userlisttypecluster"]=test["user_id"].map(mat2)

matrix = data.pivot_table(index=['user_id'], columns=['platform_family'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
x_cols = matrix.columns[1:]
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
matrix1=pd.concat([pd.DataFrame(matrix.index.values),matrix['cluster']],axis=1)
matrix1.columns=['user_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.user_id.values).to_dict()
data["userplafamcluster"] = data["user_id"].map(mat2)
test["userplafamcluster"]=test["user_id"].map(mat2)

matrix = data.pivot_table(index=['artist_id'], columns=['user_gender'], values='is_listened')
cluster = KMeans(n_clusters=6)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['artist_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.artist_id.values).to_dict()
data["artistgendcluster"] = data["artist_id"].map(mat2)
test["artistgendcluster"]=test["artist_id"].map(mat2)

matrix = data.pivot_table(index=['artist_id'], columns=['listen_type1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['artist_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.artist_id.values).to_dict()
data["artistlisttypecluster"] = data["artist_id"].map(mat2)
test["artistlisttypecluster"]=test["artist_id"].map(mat2)

matrix = data.pivot_table(index=['artist_id'], columns=['listen_type1'], values='is_listened')
cluster = KMeans(n_clusters=10)
matrix=matrix.fillna(0)
#the below line is server specific and is used to predict cluster of users and then we have mapped users on clusters
matrix['cluster'] = cluster.fit_predict(matrix[matrix.columns[1:]])
#matrix1=pd.concat([matrix.iloc[:,0],matrix['cluster']],axis=1)
opp=pd.DataFrame(matrix.index.values)
opp.reset_index(drop=True)
matrix1=pd.concat([opp,(pd.DataFrame(matrix['cluster'])).reset_index(drop=True)],axis=1,ignore_index=True).reset_index(drop=True)
matrix1.columns=['artist_id','cluster']
mat2=pd.Series(matrix1.cluster.values,index=matrix1.artist_id.values).to_dict()
data["artistlisttypecluster"] = data["artist_id"].map(mat2)
test["artistlisttypecluster"]=test["artist_id"].map(mat2)

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [10]:
from six.moves import cPickle as pickle
#data.to_pickle("data3005.pickle")
#test.to_pickle("test3005.pickle")

In [11]:
mmo3=pd.DataFrame(data['media_id'].value_counts())
mmo98=pd.DataFrame(test['media_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo3.index.values))
test['media_id'].loc[test['media_id'].isin(list(unmatched)[65:])]=-2
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.media_id.values)],axis=1)
df=pd.Series(mmo3.iloc[:,1].values,index=mmo3.iloc[:,0].values).to_dict()
data["media_id_freq"] = data["media_id"].map(df)
test["media_id_freq"] = test["media_id"].map(df)
mmo31=list(mmo3.iloc[100:,0])
data['media_id'].loc[data['media_id'].isin(mmo31)]=-1
test['media_id'].loc[test['media_id'].isin(mmo31)]=-1


mmo3=pd.DataFrame(data['album_id'].value_counts())
mmo98=pd.DataFrame(test['album_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo3.index.values))
test['album_id'].loc[test['album_id'].isin(list(unmatched)[65:])]=-2
mmo3=pd.concat([pd.DataFrame(mmo3.index.values),pd.DataFrame(mmo3.album_id.values)],axis=1)
df=pd.Series(mmo3.iloc[:,1].values,index=mmo3.iloc[:,0].values).to_dict()
data["album_id_freq"] = data["album_id"].map(df)
test["album_id_freq"] = test["album_id"].map(df)
mmo31=list(mmo3.iloc[100:,0])
data['album_id'].loc[data['album_id'].isin(mmo31)]=-1
test['album_id'].loc[test['album_id'].isin(mmo31)]=-1

mmo4=pd.DataFrame(data['artist_id'].value_counts())
mmo98=pd.DataFrame(test['artist_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo4.index.values))
test['artist_id'].loc[test['artist_id'].isin(list(unmatched)[40:])]=-2
mmo4=pd.concat([pd.DataFrame(mmo4.index.values),pd.DataFrame(mmo4.artist_id.values)],axis=1)
df=pd.Series(mmo4.iloc[:,1].values,index=mmo4.iloc[:,0].values).to_dict()
data["artist_id_freq"] = data["artist_id"].map(df)
test["artist_id_freq"] = test["artist_id"].map(df)
mmo41=list(mmo4.iloc[100:,0])
data['artist_id'].loc[data['artist_id'].isin(mmo41)]=-1
test['artist_id'].loc[test['artist_id'].isin(mmo41)]=-1

      
mmo5=pd.DataFrame(data['genre_id'].value_counts())
mmo98=pd.DataFrame(test['genre_id'].value_counts())
unmatched = set(list(mmo98.index.values))-set(list(mmo5.index.values))
test['genre_id'].loc[test['genre_id'].isin(unmatched)]=-2
mmo5=pd.concat([pd.DataFrame(mmo5.index.values),pd.DataFrame(mmo5.genre_id.values)],axis=1)
df=pd.Series(mmo5.iloc[:,1].values,index=mmo5.iloc[:,0].values).to_dict()
data["genre_id_freq"] = data["genre_id"].map(df)
test["genre_id_freq"] = test["genre_id"].map(df)
mmo51=list(mmo5.iloc[50:,0])
data['genre_id'].loc[data['genre_id'].isin(mmo51)]=-1
test['genre_id'].loc[test['genre_id'].isin(mmo51)]=-1

mmo6=pd.DataFrame(data['context_type'].value_counts())
mmo6=pd.concat([pd.DataFrame(mmo6.index.values),pd.DataFrame(mmo6.context_type.values)],axis=1)
mmo61=list(mmo6.iloc[25:,0])
data['context_type'].loc[data['context_type'].isin(mmo61)]=-1
test['context_type'].loc[test['context_type'].isin(mmo61)]=-1


mmo7=pd.DataFrame(data['release_date'].value_counts())
mmo98=pd.DataFrame(test['release_date'].value_counts())
#unmatched = set(list(mmo98.index.values))-set(list(mmo7.index.values))
#test['release_date'].loc[test['release_date'].isin(unmatched)]=-1
mmo7=pd.concat([pd.DataFrame(mmo7.index.values),pd.DataFrame(mmo7.release_date.values)],axis=1)
mmo71=list(mmo7.iloc[25:,0])
data['release_date'].loc[data['release_date'].isin(mmo71)]=-1
test['release_date'].loc[test['release_date'].isin(mmo71)]=-1
matrix=0

In [12]:
mmo3=0
mmo31=0
mmo4=0
mmo5=0
mmo6=0
mmo7=0
mmo41=0
mmo51=0
mmo61=0
mmo71=0
mmo98=0

In [13]:
target=data['is_listened']
del data['is_listened']
test2=test.iloc[:,1:] 
data.head()
test2.head()

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,...,genregendcluster,genreagedcluster,userlisttypecluster,userplafamcluster,artistgendcluster,artistlisttypecluster,media_id_freq,album_id_freq,artist_id_freq,genre_id_freq
0,50,1478104371,-1,-1,1,-1,0,0,542,1,...,1,2,2,0,3.0,8.0,6.0,20.0,1111.0,79158
1,2744,1479317140,-1,-1,1,-1,0,0,307,1,...,1,2,0,0,3.0,8.0,441.0,1363.0,5536.0,84702
2,2744,1479546361,-1,-1,1,-1,0,0,307,1,...,1,2,5,0,3.0,8.0,441.0,1363.0,5536.0,84702
3,2744,1478457729,-1,-1,1,-1,2,1,265,1,...,1,2,2,2,3.0,8.0,176.0,1363.0,5536.0,84702
4,2744,1480448560,-1,-1,1,-1,2,1,356,1,...,1,2,6,1,3.0,8.0,15.0,1363.0,5536.0,84702


In [14]:
data=pd.concat([data,test2],axis=0)
data=data.fillna(0)

In [15]:
#MediaDuration
data['media_duration'].loc[(data['media_duration']>=500) ]=0
#data['media_duration'].loc[(data['media_duration']>=400 )&( data['media_duration']<500) ]=1
#data['media_duration'].loc[(data['media_duration']>=300 )&( data['media_duration']<500) ]=2
data['media_duration'].loc[(data['media_duration']>=250 )&( data['media_duration']<500) ]=-3
data['media_duration'].loc[(data['media_duration']>=150 )&( data['media_duration']<250) ]=-4
data['media_duration'].loc[(data['media_duration']>=30 )&( data['media_duration']<150) ]=-5
data['media_duration'].loc[(data['media_duration'] <30) &( data['media_duration']>0) ]=-6

#diff
data['diff'].loc[(data['diff']>=6000) ]=0
#data['diff'].loc[(data['diff']>=20000 )&( data['diff']<50000) ]=1
#data['diff'].loc[(data['diff']>=10000 )&( data['diff']<20000) ]=2
#data['diff'].loc[(data['diff']>=8000 )&( data['diff']<10000) ]=3
#data['diff'].loc[(data['diff']>=6000 )&( data['diff']<8000) ]=4
data['diff'].loc[(data['diff']>=2000 )&( data['diff']<6000) ]=-5
data['diff'].loc[(data['diff']>=1000 )&( data['diff']<2000) ]=-6
data['diff'].loc[(data['diff']>=200 )&( data['diff']<1000) ]=-7
data['diff'].loc[(data['diff']>=100 )&( data['diff']<200) ]=-8
#data['diff'].loc[(data['diff']>=100 )&( data['diff']<300) ]=9
data['diff'].loc[(data['diff']>=50 )&( data['diff']<100) ]=-10
data['diff'].loc[(data['diff']>=25 )&( data['diff']<50) ]=-11
data['diff'].loc[(data['diff']>=10 )&( data['diff']<25) ]=-12
data['diff'].loc[(data['diff']<10 ) &( data['diff']>0)]=-13

#B
#data['B'].loc[(data['B']==0) ]=0
data['B'].loc[(data['B']>=0 )&( data['B']<0.5) ]=-1
data['B'].loc[(data['B']>=0.5 )&( data['B']<0.7) ]=-2
#data['B'].loc[(data['B']>0.4 )&( data['B']<=0.5) ]=3
#data['B'].loc[(data['B']>0.5 )&( data['B']<=0.6) ]=4
#data['B'].loc[(data['B']>0.6 )&( data['B']<=0.7) ]=5
#data['B'].loc[(data['B']>0.7 )&( data['B']<=0.8) ]=6
data['B'].loc[(data['B']>=0.7 )&( data['B']<=1) ]=-7
#data['B'].loc[(data['B']>0.9 )&( data['B']<1) ]=8
#data['B'].loc[(data['B']==1 ) ]=9

#C
#data['C'].loc[(data['C']==0) ]=0
data['C'].loc[(data['C']>=0 )&( data['C']<0.5) ]=-1
data['C'].loc[(data['C']>=0.5 )&( data['C']<0.9) ]=-2
#data['C'].loc[(data['C']>0.4 )&( data['C']<=0.5) ]=3
#data['C'].loc[(data['C']>0.5 )&( data['C']<=0.6) ]=4
#data['C'].loc[(data['C']>0.6 )&( data['C']<=0.7) ]=5
#data['C'].loc[(data['C']>0.7 )&( data['C']<=0.8) ]=6
#data['C'].loc[(data['C']>=0.8 )&( data['C']<=0.9) ]=7
data['C'].loc[(data['C']>=0.9 )&( data['C']<=1) ]=-8
#data['C'].loc[(data['C']==1 ) ]=9

#D
#data['D'].loc[(data['D']==0) ]=0
data['D'].loc[(data['D']>=0 )&( data['D']<0.5) ]=-1
data['D'].loc[(data['D']>=0.5 )&( data['D']<=0.95) ]=-2
#data['D'].loc[(data['D']>0.4 )&( data['D']<=0.5) ]=3
#data['D'].loc[(data['D']>0.5 )&( data['D']<=0.6) ]=4
#data['D'].loc[(data['D']>0.6 )&( data['D']<=0.8) ]=5
#data['D'].loc[(data['D']>0.7 )&( data['D']<=0.9) ]=6
data['D'].loc[(data['D']>0.95 )&( data['D']<=1) ]=-7
#data['D'].loc[(data['D']==1 ) ]=8

#E
#data['E'].loc[(data['E']==0) ]=0
data['E'].loc[(data['E']>0 )&( data['E']<=0.5) ]=-1
data['E'].loc[(data['E']>0.5 )&( data['E']<=0.8) ]=-2
#data['E'].loc[(data['E']>0.4 )&( data['E']<=0.5) ]=3
#data['E'].loc[(data['E']>0.5 )&( data['E']<=0.6) ]=4
#data['E'].loc[(data['E']>0.6 )&( data['E']<=0.8) ]=5
#data['E'].loc[(data['E']>0.8 )&( data['E']<=0.9) ]=6
data['E'].loc[(data['E']>0.8 )&( data['E']<=1) ]=-7
#data['E'].loc[(data['E']==1 ) ]=8

#F
#data['F'].loc[(data['F']==0) ]=0
data['F'].loc[(data['F']>=0 )&( data['F']<=0.3) ]=-1
data['F'].loc[(data['F']>0.3)&( data['F']<=0.5) ]=-2
data['F'].loc[(data['F']>0.5 )&( data['F']<=0.8) ]=-3
#data['F'].loc[(data['F']>0.3 )&( data['F']<=0.4) ]=4
#data['F'].loc[(data['F']>0.4 )&( data['F']<=0.5) ]=5
#data['F'].loc[(data['F']>0.5 )&( data['F']<=0.6) ]=6
#data['F'].loc[(data['F']>0.6 )&( data['F']<=0.7) ]=7
#data['F'].loc[(data['F']>0.7 )&( data['F']<=0.8) ]=8
data['F'].loc[(data['F']>0.8 )&( data['F']<=1) ]=-9
#data['F'].loc[(data['F']>0.9 )&( data['F']<1) ]=10
#data['F'].loc[(data['F']==1 ) ]=11

#G
#data['G'].loc[(data['G']==0) ]=0
data['G'].loc[(data['G']>=0 )&( data['G']<=0.5) ]=-1
data['G'].loc[(data['G']>0.5 )&( data['G']<=0.7) ]=-2
data['G'].loc[(data['G']>0.7 )&( data['G']<=0.8) ]=-3
#data['G'].loc[(data['G']>0.5 )&( data['G']<=0.6) ]=4
#data['G'].loc[(data['G']>0.6 )&( data['G']<=0.7) ]=5
#data['G'].loc[(data['G']>0.7 )&( data['G']<=0.8) ]=6
#data['G'].loc[(data['G']>0.8 )&( data['G']<=0.9) ]=7
data['G'].loc[(data['G']>0.8 )&( data['G']<=1) ]=-8
#data['G'].loc[(data['G']==1 ) ]=9

In [16]:
dummie1 = pd.get_dummies(data['context_type'], prefix='context_type', prefix_sep='_')
dummie2 = pd.get_dummies(data['platform_name'], prefix='platform_name', prefix_sep='_')
dummie3 = pd.get_dummies(data['platform_family'], prefix='platform_family', prefix_sep='_')
dummie4 = pd.get_dummies(data['listen_type'], prefix='listen_type', prefix_sep='_')
dummie5 = pd.get_dummies(data['user_gender'], prefix='user_gender', prefix_sep='_')
dummie6 = pd.get_dummies(data['genre_id'], prefix='genre_id', prefix_sep='_')
dummie7 = pd.get_dummies(data['album_id'], prefix='album_id', prefix_sep='_')
dummie8 = pd.get_dummies(data['release_date'], prefix='album_id', prefix_sep='_')

dummie16 = pd.get_dummies(data['media_id'], prefix='media_id', prefix_sep='_')
#dummie9 = pd.get_dummies(data['user_id'], prefix='user_id', prefix_sep='_',sparse=True)
dummie9 = pd.get_dummies(data['usergencluster'], prefix='usergencluster', prefix_sep='_')
dummie10 = pd.get_dummies(data['artist_id'], prefix='artist_id', prefix_sep='_')
dummie11 = pd.get_dummies(data['usermedcluster'], prefix='usermedcluster', prefix_sep='_')
dummie12= pd.get_dummies(data['useralbcluster'], prefix='useralbcluster', prefix_sep='_')
dummie13= pd.get_dummies(data['userartcluster'], prefix='userartcluster', prefix_sep='_')
dummie14= pd.get_dummies(data['userdatecluster'], prefix='userdatecluster', prefix_sep='_')
dummie15= pd.get_dummies(data['usertimecluster'], prefix='usertimecluster', prefix_sep='_')
dummie26= pd.get_dummies(data['useragecluster'], prefix='useragecluster', prefix_sep='_')

dummie25= pd.get_dummies(data['user_age'], prefix='user_age', prefix_sep='_')
dummie17= pd.get_dummies(data['usercontcluster'], prefix='usercontcluster', prefix_sep='_')
dummie18= pd.get_dummies(data['userplatcluster'], prefix='userplatcluster', prefix_sep='_')
dummie19= pd.get_dummies(data['userlisttypecluster'], prefix='userlisttypecluster', prefix_sep='_')
dummie27= pd.get_dummies(data['userplafamcluster'], prefix='userplafamcluster', prefix_sep='_')
dummie28= pd.get_dummies(data['usergendcluster'], prefix='usergendcluster', prefix_sep='_')
dummie29= pd.get_dummies(data['genreagecluster'], prefix='genreagecluster', prefix_sep='_')
dummie30= pd.get_dummies(data['genrecontcluster'], prefix='genrecontcluster', prefix_sep='_')
dummie31= pd.get_dummies(data['genrelistcluster'], prefix='genrelistcluster', prefix_sep='_')
dummie32= pd.get_dummies(data['genregendcluster'], prefix='genregendcluster', prefix_sep='_')
dummie33= pd.get_dummies(data['genreagedcluster'], prefix='genreagedcluster', prefix_sep='_')
dummie34= pd.get_dummies(data['artistgendcluster'], prefix='artistgendcluster', prefix_sep='_')
dummie35= pd.get_dummies(data['artistlisttypecluster'], prefix='artistlisttypecluster', prefix_sep='_')
dummie36= pd.get_dummies(data['artistlisttypecluster'], prefix='artistlisttypecluster', prefix_sep='_')


dummie37= pd.get_dummies(data['B'], prefix='B', prefix_sep='_')
dummie38= pd.get_dummies(data['C'], prefix='C', prefix_sep='_')
dummie39= pd.get_dummies(data['D'], prefix='D', prefix_sep='_')
dummie40= pd.get_dummies(data['E'], prefix='E', prefix_sep='_')
dummie41= pd.get_dummies(data['F'], prefix='F', prefix_sep='_')
dummie42= pd.get_dummies(data['G'], prefix='G', prefix_sep='_')
dummie43= pd.get_dummies(data['diff'], prefix='diff', prefix_sep='_')
#dummie24= pd.get_dummies(data['media_duration'], prefix='media_duration', prefix_sep='_')#


In [17]:
data=pd.concat([data,dummie1,dummie2,dummie3,dummie4,dummie5,dummie6,dummie7,dummie8,dummie9,dummie10,dummie11,dummie12,dummie13,dummie14,dummie15,dummie17,dummie16,dummie18,dummie19,dummie16,dummie25,dummie26,dummie28,dummie29,dummie30,dummie31,dummie32,dummie33,dummie34,dummie35,dummie36,dummie37,dummie38,dummie39,dummie40,dummie41,dummie42,dummie43],axis=1)
data.head()

,B,C,D,E,F,G,H,album_id,album_id_1,album_id_freq,...,G_-1.0,diff_-13,diff_-12,diff_-11,diff_-10,diff_-8,diff_-7,diff_-6,diff_-5,diff_0
0,-2.0,-1.0,-1.0,0.0,-3.0,-3.0,0.706600,-1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
1,-2.0,-8.0,-7.0,-1.0,-3.0,-8.0,0.626347,-1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
2,-7.0,-8.0,-7.0,-2.0,-9.0,-3.0,0.706600,-1,-1,1.0,...,0,0,0,0,0,0,1,0,0,0
3,-2.0,-1.0,-1.0,-2.0,-3.0,-2.0,0.716894,-1,-1,1.0,...,0,0,0,0,0,0,0,0,1,0
4,-2.0,-2.0,-2.0,-7.0,-9.0,-3.0,0.667209,-1,-1,34.0,...,0,0,0,0,0,0,0,0,1,0


In [18]:
del dummie1
del dummie2
del dummie3
del dummie4
del dummie5
del dummie6
del dummie7
del dummie8
del dummie9
del dummie10
del dummie11
del dummie12
del dummie13
del dummie14
del dummie15
del dummie16
del dummie17
del dummie18
del dummie19
del dummie30
del dummie31
del dummie32
del dummie33
del dummie34
del dummie20
del dummie25
del dummie26

NameError: name 'dummie20' is not defined

In [19]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import scipy.sparse as sps

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
y = np.asarray(target)

In [31]:
data.columns.get_loc("context_type_-1")

55

In [22]:
predictors = data.iloc[:,35:].as_matrix()# assign feature dataframe to predictors removing the target column
#target1 = to_categorical(target)# assign target dataframe to y
n_cols = predictors.shape[1]
predictors.shape

(7578752, 1278)

In [46]:
data.columns.get_loc('B_-1.0')

1285

In [23]:
x = sps.csr_matrix(predictors[:3000000].astype(np.float))
x

<3000000x1278 sparse matrix of type '<type 'numpy.float64'>'
	with 167164191 stored elements in Compressed Sparse Row format>

In [32]:
fm = pylibfm.FM(verbose=True, task="classification", initial_learning_rate=0.0001, learning_rate_schedule="optimal")
start_time = datetime.datetime.now()
fm.fit(x[:,20:],y[:3000000])
print(datetime.datetime.now()-start_time)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training log loss: 0.48544
0:10:01.660641


In [47]:
fm1 = pylibfm.FM(verbose=True, task="classification", initial_learning_rate=0.0001, learning_rate_schedule="optimal")
start_time = datetime.datetime.now()
fm1.fit(x[:,20:1250],y[:3000000])
print(datetime.datetime.now()-start_time)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training log loss: 0.51327
0:03:51.792993


In [48]:
X_test = sps.csr_matrix(predictors[7558834:,20:1250].astype(np.float))

In [49]:
preds = fm.predict(X_test)

In [50]:
preds

array([ 0.53651935,  0.42769682,  0.57127618, ...,  0.50354658,
        0.4041672 ,  0.59796232])

In [51]:
submission = pd.DataFrame(preds, columns=['is_listened'])
submission.to_csv('libffm_full_4.csv')